In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import geatpy as ea
import pandas as pd
from numpy import exp
from scipy.optimize import linprog
import time
import FragStatsPy.frag_model as fspy
import geopandas as gpd
import rasterio as rio
import shutil
import joblib
import sys
import os

In [ ]:
isa_filepath = r'E:\Paper4_new2\d_isa_raster\isa.tif'

In [ ]:
with rio.open(isa_filepath) as src1:
    profile = src1.profile
    width, height = src1.width, src1.height
    ISA_2d = src1.read(1)
    nodata_value = src1.nodata
print("Nodata value：", nodata_value)
print('Figure width,height：', height, width,)
print(ISA_2d.shape)
not_Nodata_count = len(ISA_2d[ISA_2d != nodata_value])

In [ ]:
ISA_1d = ISA_2d.flatten()
isa_optim = ISA_1d[np.where(ISA_1d != nodata_value)]
ISA_area_SUM = isa_optim.sum()
print(isa_optim)
print(isa_optim.shape)

In [ ]:
inter2 = np.where(ISA_2d == nodata_value, True, False)
print(inter2.shape)

In [ ]:
filepath_dem = 'E:/Paper4_new2/f_dem_slope_guanwang/'
file_names = ['dem.tif', 'guanwang.tif', 'slope.tif', 'water.tif']

In [ ]:
factor_other = np.empty((len(file_names), not_Nodata_count))
factor_other.shape

In [ ]:
factor_other = np.empty((len(file_names), not_Nodata_count))

for i, file_name in enumerate(file_names):
    file_path = filepath_dem + file_name
    with rio.open(file_path) as src:
        data = src.read(1)
        data[inter2] = np.array([-999]).astype(data.dtype)
        data = data.flatten()
        factor_other[i, :] = data[data != -999]
Fac = np.transpose(factor_other)
factors_dem = (Fac - Fac.min(axis=0)) / (Fac.max(axis=0) - Fac.min(axis=0))
print(factors_dem.shape)
print(factors_dem)

In [ ]:
filepath_fca = r'E:\Paper4_new2\h_couple_code\unnamed1250.fca'
filepath_Chongfenlei = r'E:\Paper4_new2\h_couple_code\isa_ChongFenLei.tif'
filepath_tifmw1 = 'E:/Paper4_new2/h_couple_code/isa_chongfenlei.tif_mw1/'
landscape_name = ['pland_1.tif', 'pland_2.tif', 'pland_3.tif', 'pland_4.tif',
                  'pland_5.tif',
                  'ai_1.tif', 'ai_2.tif', 'ai_3.tif', 'ai_4.tif', 'ai_5.tif',
                  'para_mn_1.tif', 'para_mn_2.tif', 'para_mn_3.tif',
                  'para_mn_4.tif', 'para_mn_5.tif']

In [ ]:
chongfenlei = np.zeros_like(ISA_2d)
chongfenlei[ISA_2d == nodata_value] = nodata_value
chongfenlei[(ISA_2d >= 0) & (ISA_2d <= 0.2)] = 1
chongfenlei[(ISA_2d > 0.2) & (ISA_2d <= 0.4)] = 2
chongfenlei[(ISA_2d > 0.4) & (ISA_2d <= 0.6)] = 3
chongfenlei[(ISA_2d > 0.6) & (ISA_2d <= 0.8)] = 4
chongfenlei[ISA_2d > 0.8] = 5
with rio.open(filepath_Chongfenlei, 'w', **profile) as dst:
    dst.write(chongfenlei, 1)
model = fspy.FragModel(filepath_fca)
model.load_landscape_layer(filepath_Chongfenlei)
model.set_output_base_path(r'E:\Paper4_new2\shanchu')
model.run_model()

folder_path = filepath_tifmw1
w = len(landscape_name)

landscape = np.empty((not_Nodata_count, w+1))

for i, file_name in enumerate(landscape_name):
    file_path = folder_path + file_name
    with rio.open(file_path) as src:
        data = src.read(1)
        data[inter2] = nodata_value
        data = data.flatten()
        landscape[:, i] = data[data != nodata_value]

isa_density = ISA_2d[np.where(ISA_2d != nodata_value)]
landscape[:, w] = isa_density

landscape[landscape == -999] = np.nan
min_vals = np.nanmin(landscape, axis=0)
max_vals = np.nanmax(landscape, axis=0)

factors_nor = (landscape - min_vals) / (max_vals - min_vals)
factors_nor[np.isnan(factors_nor)] = 0

factors_merged = np.concatenate((factors_dem, factors_nor), axis=1)
print(factors_merged.shape)

In [ ]:
lightgbm_model_file_path = r'E:\Paper4_new2\g_MLearning\lightgbm01.pkl'
filepath_risk = r'E:\Paper4_new2\h_couple_code\raster\Ori_Risk_250.tif'

In [ ]:
model_path = lightgbm_model_file_path
lgb_clf_best1 = joblib.load(model_path)
pred = lgb_clf_best1.predict_proba(factors_merged)[:, 1]
ori_risk_sum = pred.sum()

In [ ]:
ISA_1d[np.where(ISA_1d != nodata_value)] = pred
ISA_2d_1 = ISA_1d.reshape(height, width)
with rio.open(filepath_risk, 'w', **profile) as dst:
    dst.write(ISA_2d_1, 1)

In [ ]:
filepath_cfl = 'E:/Paper4_new2/h_couple_code/objective_function/cfl.tif'
filepath_fca = r'E:\Paper4_new2\h_couple_code\unnamed1250.fca'
filepath_landscape = 'E:/Paper4_new2/h_couple_code/objective_function/cfl.tif_mw1/'

landscape_name = ['pland_1.tif', 'pland_2.tif', 'pland_3.tif', 'pland_4.tif',
                  'pland_5.tif',
                  'ai_1.tif', 'ai_2.tif', 'ai_3.tif', 'ai_4.tif', 'ai_5.tif',
                  'para_mn_1.tif', 'para_mn_2.tif', 'para_mn_3.tif',
                  'para_mn_4.tif', 'para_mn_5.tif']
filepath_delete_mwl = 'E:/Paper4_new2/h_couple_code/objective_function/cfl.tif_mw1'

In [ ]:
def objective(Vars):
    for m in range(0, NIND):
        array1 = Vars[m, :]/xiaoshu
        ISA_1d[np.where(ISA_1d != nodata_value)] = array1
        ISA_2d_1 = ISA_1d.reshape(height, width)
        chongfenlei = np.zeros_like(ISA_2d_1)
        chongfenlei[ISA_2d_1 == nodata_value] = nodata_value
        chongfenlei[(ISA_2d_1 >= 0) & (ISA_2d_1 <= 0.2)] = 1
        chongfenlei[(ISA_2d_1 > 0.2) & (ISA_2d_1 <= 0.4)] = 2
        chongfenlei[(ISA_2d_1 > 0.4) & (ISA_2d_1 <= 0.6)] = 3
        chongfenlei[(ISA_2d_1 > 0.6) & (ISA_2d_1 <= 0.8)] = 4
        chongfenlei[ISA_2d_1 > 0.8] = 5
        with rio.open(filepath_cfl, 'w', **profile) as dst:
            dst.write(chongfenlei, 1)

        model = fspy.FragModel(filepath_fca)
        old_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        model.load_landscape_layer(filepath_cfl)
        model.set_output_base_path(r'F:\01')
        model.run_model()
        sys.stdout = old_stdout
        w = len(landscape_name)
        landscape = np.empty((not_Nodata_count, w+1))
        for i, file_name in enumerate(landscape_name):
            file_path = filepath_landscape + file_name
            with rio.open(file_path) as src:
                data = src.read(1)
                data[inter2] = nodata_value
                data = data.flatten()
                landscape[:, i] = data[data != nodata_value]

        isa_density = ISA_2d_1[np.where(ISA_2d_1 != nodata_value)]
        landscape[:, w] = isa_density
        landscape[landscape == -999] = np.nan
        factors_nor = (landscape - min_vals) / (max_vals - min_vals)
        factors_nor[np.isnan(factors_nor)] = 0
        factors_merged = np.concatenate((factors_dem, factors_nor), axis=1)
        model_path = lightgbm_model_file_path
        lgb_clf_best1 = joblib.load(model_path)
        pred = lgb_clf_best1.predict_proba(factors_merged)[:, 1]
        obj[m, :] = pred
        shutil.rmtree(filepath_delete_mwl)
    return obj

In [ ]:
Encoding = 'RI'
NIND = 500
maximum_iterations = 600
bianyi_coefficient = 0.1
jiaocha_coefficient = 0.7
log = 1
filepath_result = 'E:/Paper4_new2/i_optim_result/NIND'
ISA_SUM_per = 0.95
xia_ISA_per = 0.7
shang_ISA_per = 0.7
xiaoshu = 100

In [ ]:
ori_var1 = (isa_optim*xiaoshu).round().astype(int)
print(ori_var1.sum())
print(ori_var1[0:20])

In [ ]:
ISA_SUM_shang = ori_var1.sum()
ISA_SUM_xia = int(ISA_SUM_shang*ISA_SUM_per)

In [ ]:
obj = np.empty((NIND, not_Nodata_count))

In [ ]:
filepath_green_water = r'E:\Paper4_new2\h_couple_code\banArea_new1\green_water.tif'
filtpath_road = r'E:\Paper4_new2\h_couple_code\banArea_new1\road.tif'

In [ ]:
with rio.open(filepath_green_water) as src1:
    profile = src1.profile
    width, height = src1.width, src1.height
    green_2d = src1.read(1)
    nodata_value = src1.nodata
green_1d = green_2d.flatten()
not_Nodata_count = len(green_2d[green_2d != nodata_value])

In [ ]:
with rio.open(filtpath_road) as src1:
    profile = src1.profile
    width, height = src1.width, src1.height
    road_2d = src1.read(1)
    nodata_value = src1.nodata

road_1d = road_2d.flatten()
not_Nodata_count = len(road_2d[road_2d != nodata_value])
print("Nodata value：", nodata_value)
print('width,height：', height, width)

In [ ]:
green1 = green_1d[np.where(green_1d != nodata_value)]
#green2 = (green1*100).round().astype(int)
sj_green = 1-green1
print('-------------------------------------------')
road1 = road_1d[np.where(road_1d != nodata_value)]
print('---------------------------')
m = [sj_green[i] - road1[i] for i in range(len(sj_green))]

In [ ]:
sj = (isa_optim*shang_ISA_per+1-shang_ISA_per).tolist()
xj = (isa_optim*shang_ISA_per).tolist()
shangjie1 = [sj[i] if sj[i] < sj_green[i] else sj_green[i] for i in range(len(sj))]
xiajie1 = [road1[i] if xj[i] < road1[i] else xj[i] for i in range(len(xj))]
print(shangjie1[0:10])
print(sum(shangjie1))
print('--------------------------')
print(xiajie1[0:10])
print(sum(xiajie1))
print('---------------------------')
chazhi = [shangjie1[i] - xiajie1[i] for i in range(len(sj_green))]
print(min(chazhi))

In [ ]:
shangjie = [round(num * 100) for num in shangjie1]
xiajie = [round(num * 100) for num in xiajie1]
chazhi = [shangjie[i] - xiajie[i] for i in range(len(sj_green))]

In [ ]:
class MyProblem(ea.Problem):
    def __init__(self):
        name = 'BNH'
        M = 2
        maxormins = [1] * M
        Dim = not_Nodata_count
        varTypes = [1] * Dim
        lb = xiajie
        ub = shangjie
        lbin = [1] * Dim
        ubin = [1] * Dim
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)

    def aimFunc(self, pop):
        Vars = pop.Phen
        obj = objective(Vars)
        f1 = np.sum(obj, axis=1)
        f2 = np.absolute(np.sum(Vars, axis=1) - 875990)
        pop.ObjV = np.hstack((f1[:, np.newaxis], f2[:, np.newaxis]))

In [ ]:
# -*- coding: utf-8 -*-
problem = MyProblem()
#
Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges, problem.borders)
population = ea.Population(Encoding, Field, NIND)
"""=========================Algorithm parameter setting============================"""
myAlgorithm = ea.moea_NSGA2_templet(problem, population)
myAlgorithm.mutOper.Pm = bianyi_coefficient
myAlgorithm.recOper.XOVR = jiaocha_coefficient
myAlgorithm.MAXGEN = maximum_iterations
myAlgorithm.logTras = log
myAlgorithm.verbose = True
myAlgorithm.drawing = 1
"""========================================
"""
[NDSet, population] = myAlgorithm.run()
NDSet.save(filepath_result)
"""===========================output result========================"""
print('time：%s s' % myAlgorithm.passTime)
print('Number of non-dominant individuals：%d 个' % NDSet.sizes) if NDSet.sizes !=0 else print('No feasible solution was found！')
if myAlgorithm.log is not None and NDSet.sizes != 0:
    print('GD', myAlgorithm.log['gd'][-1])
    print('IGD', myAlgorithm.log['igd'][-1])
    print('HV', myAlgorithm.log['hv'][-1])
    print('Spacing', myAlgorithm.log['spacing'][-1])
"""======================Evolutionary process index tracking analysis=================="""
metricName = [['igd'], ['hv']]
Metrics = np.array([myAlgorithm.log[metricName[i][0]] for i in range(len(metricName))]).T
ea.trcplot(Metrics, labels=metricName, titles=metricName)

In [ ]:
import pandas as pd
df_variable = pd.read_csv(r'E:\Paper4_new2\shanchu\Phen.csv', header = None)
df_ObjV = pd.read_csv(r'E:\Paper4_new2\shanchu\ObjV.csv', header = None)
df_ObjV.columns = ['f1_risk', 'f2_isa_sum']
df_ov = pd.concat([df_ObjV, df_variable], axis=1)
df_ov1 = df_ov.loc[df_ov.f2_isa_sum == df_ov['f2_isa_sum'].min()]
pp = df_ov1.values
pp = pp[0, 2:]
pp = pp/100

In [ ]:
filepath_isa_optimization = r'E:\Paper4_new2\i_optim_result\isa_optim_result.tif'

In [ ]:
ISA_1d[np.where(ISA_1d != nodata_value)] = pp
ISA_2d = ISA_1d.reshape(height, width)

In [ ]:
with rio.open(filepath_isa_optimization, 'w', **profile) as dst:
    dst.write(ISA_2d, 1)